In [ ]:
# %pip install torch torchaudio pytorch-lightning transformers datasets

In [1]:
# import os
# import random
# import shutil
# from pathlib import Path
# from tqdm import tqdm
# from multiprocessing import Pool
# from functools import partial

# def copy_file(args):
#     """Helper function to copy a single file and handle errors."""
#     src, dest = args
#     try:
#         shutil.copy(src, dest)
#         return None  # Success
#     except (OSError, shutil.Error) as e:
#         return f"Error copying {src}: {e}"

# def split_audio_dataset(base_real, base_fake, output_base, train_ratio=0.8, val_ratio=0.1, num_processes=4):
#     # Supported audio extensions
#     valid_extensions = {'.wav', '.mp3', '.flac', '.ogg', '.m4a'}

#     # Create output directories
#     for split in ["train", "val", "test"]:
#         for label in ["real", "fake"]:
#             Path(os.path.join(output_base, split, label)).mkdir(parents=True, exist_ok=True)

#     def split_and_copy(files, label):
#         random.shuffle(files)
#         total = len(files)
#         if total == 0:
#             print(f"Warning: No files found for {label}.")
#             return

#         train_end = int(train_ratio * total)
#         val_end = train_end + int(val_ratio * total)

#         splits = {
#             "train": files[:train_end],
#             "val": files[train_end:val_end],
#             "test": files[val_end:]
#         }

#         # Copy files using multiprocessing
#         for split, file_list in splits.items():
#             print(f"Copying {len(file_list)} files to {split}/{label}...")
#             # Prepare source-destination pairs
#             dest_dir = os.path.join(output_base, split, label)
#             copy_tasks = [(f, os.path.join(dest_dir, os.path.basename(f))) for f in file_list]
            
#             # Use multiprocessing Pool
#             with Pool(processes=num_processes) as pool:
#                 # Use tqdm for progress tracking
#                 errors = list(tqdm(
#                     pool.imap_unordered(copy_file, copy_tasks),
#                     total=len(copy_tasks),
#                     desc=f"{split}/{label}",
#                     unit="file"
#                 ))
            
#             # Log any errors
#             for error in errors:
#                 if error:
#                     print(error)

#         return splits

#     # Collect files with valid extensions
#     real_files = [
#         os.path.join(base_real, f) for f in os.listdir(base_real)
#         if os.path.splitext(f)[1].lower() in valid_extensions
#     ]
#     fake_files = [
#         os.path.join(base_fake, f) for f in os.listdir(base_fake)
#         if os.path.splitext(f)[1].lower() in valid_extensions
#     ]

#     # Verify input directories
#     if not real_files:
#         print(f"Warning: No valid audio files found in {base_real}")
#     if not fake_files:
#         print(f"Warning: No valid audio files found in {base_fake}")

#     print(f"Found {len(real_files)} real and {len(fake_files)} fake files.")

#     # Perform split and copy
#     real_splits = split_and_copy(real_files, "real")
#     fake_splits = split_and_copy(fake_files, "fake")

#     # Log split sizes
#     if real_splits and fake_splits:
#         for split in ["train", "val", "test"]:
#             real_count = len(real_splits[split])
#             fake_count = len(fake_splits[split])
#             print(f"{split.capitalize()} split: {real_count} real, {fake_count} fake")

#     print("Done splitting into train, val, test.")

# # Usage
# if __name__ == "__main__":
#     split_audio_dataset(
#         base_real="/teamspace/studios/this_studio/audio_detect/dataset/processed_data_2s/real",
#         base_fake="/teamspace/studios/this_studio/audio_detect/dataset/processed_data_2s/fake",
#         output_base="/teamspace/studios/this_studio/audio_detect/dataset/split_data",
#         num_processes=4  # Adjust based on your CPU cores
#     )


In [3]:
# import os

# def count_files(directory):
#     return sum([len(files) for _, _, files in os.walk(directory)])

# train_real_dir = "/teamspace/studios/this_studio/audio_detect/dataset/split_data/train/real"
# val_real_dir = "/teamspace/studios/this_studio/audio_detect/dataset/split_data/val/real"
# test_real_dir = "/teamspace/studios/this_studio/audio_detect/dataset/split_data/test/real"
# train_fake_dir = "/teamspace/studios/this_studio/audio_detect/dataset/split_data/train/fake"
# val_fake_dir = "/teamspace/studios/this_studio/audio_detect/dataset/split_data/val/fake"
# test_fake_dir = "/teamspace/studios/this_studio/audio_detect/dataset/split_data/test/fake"

# print("Train real files:", count_files(train_real_dir))
# print("Validation real files:", count_files(val_real_dir))
# print("Test real files:", count_files(test_real_dir))

# print("Train fake files:", count_files(train_fake_dir))
# print("Validation fake files:", count_files(val_fake_dir))
# print("Test fake files:", count_files(test_fake_dir))

In [4]:
# import os
# import torch
# import torchaudio
# import random
# from torch.utils.data import Dataset, DataLoader, ConcatDataset
# from transformers import Wav2Vec2Processor

# # Check for GPU availability
# if not torch.cuda.is_available():
#     print("Warning: No GPU detected. Training on CPU will be significantly slower.")

# # Custom dataset for real/fake audio
# class AudioDataset(Dataset):
#     def __init__(self, data_dir, processor, label, extensions=(".wav", ".mp3", ".flac", ".ogg", ".m4a")):
#         """
#         Initialize the dataset for real or fake audio files.
        
#         Args:
#             data_dir (str): Directory containing audio files.
#             processor (Wav2Vec2Processor): Processor for Wav2Vec2 model.
#             label (int): Label for the class (0 for real, 1 for fake).
#             extensions (tuple): File extensions to filter (default: '.wav', '.mp3', '.flac', '.ogg', '.m4a').
#         """
#         self.files = [
#             os.path.join(data_dir, f) for f in os.listdir(data_dir)
#             if f.lower().endswith(extensions) and os.path.isfile(os.path.join(data_dir, f))
#         ]
#         self.processor = processor
#         self.label = label
#         if not self.files:
#             print(f"Warning: No valid audio files found in {data_dir}")

#     def __len__(self):
#         return len(self.files)

#     def __getitem__(self, idx):
#         file_path = self.files[idx]
#         try:
#             waveform, sr = torchaudio.load(file_path)
#             # Ensure 16kHz sampling rate (Wav2Vec2 requirement)
#             if sr != 16000:
#                 resampler = torchaudio.transforms.Resample(sr, 16000)
#                 waveform = resampler(waveform)
#             # Ensure mono audio (Wav2Vec2 expects 1D input)
#             if waveform.shape[0] > 1:
#                 waveform = torch.mean(waveform, dim=0, keepdim=True)
#             waveform = waveform.squeeze(0)  # Remove channel dim
#             # Process waveform with Wav2Vec2Processor
#             inputs = self.processor(
#                 waveform, sampling_rate=16000, return_tensors="pt", padding=True
#             )
#             return {
#                 "input_values": inputs.input_values.squeeze(0),
#                 "attention_mask": inputs.attention_mask.squeeze(0),
#                 "labels": torch.tensor(self.label, dtype=torch.long)
#             }
#         except Exception as e:
#             print(f"Error loading {file_path}: {e}")
#             # Return a random item to avoid infinite recursion
#             return self.__getitem__(random.randint(0, self.__len__() - 1))

# # Create DataLoaders
# def create_dataloaders(train_real_dir, train_fake_dir, val_real_dir, val_fake_dir, processor, batch_size=16):
#     """
#     Create DataLoaders for training and validation datasets, optimized for GPU.
    
#     Args:
#         train_real_dir (str): Directory for training real audio.
#         train_fake_dir (str): Directory for training fake audio.
#         val_real_dir (str): Directory for validation real audio.
#         val_fake_dir (str): Directory for validation fake audio.
#         processor (Wav2Vec2Processor): Processor for Wav2Vec2 model.
#         batch_size (int): Batch size for DataLoader (default: 16, adjust to 8 for smaller GPUs).
    
#     Returns:
#         tuple: (train_loader, val_loader)
#     """
#     # Initialize datasets
#     train_real = AudioDataset(train_real_dir, processor, label=0)
#     train_fake = AudioDataset(train_fake_dir, processor, label=1)
#     val_real = AudioDataset(val_real_dir, processor, label=0)
#     val_fake = AudioDataset(val_fake_dir, processor, label=1)

#     # Check for empty datasets
#     for dataset, name in [
#         (train_real, "train_real"), (train_fake, "train_fake"),
#         (val_real, "val_real"), (val_fake, "val_fake")
#     ]:
#         if len(dataset) == 0:
#             raise ValueError(f"No audio files found in {name} directory")

#     # Combine real and fake datasets
#     train_dataset = ConcatDataset([train_real, train_fake])
#     val_dataset = ConcatDataset([val_real, val_fake])

#     # Create DataLoaders with GPU optimizations
#     train_loader = DataLoader(
#         train_dataset,
#         batch_size=batch_size,
#         shuffle=True,
#         num_workers=4,  # Parallel data loading for GPU
#         pin_memory=True  # Fast data transfer to GPU
#     )
#     val_loader = DataLoader(
#         val_dataset,
#         batch_size=batch_size,
#         shuffle=False,
#         num_workers=4,  # Parallel data loading for GPU
#         pin_memory=True  # Fast data transfer to GPU
#     )
#     return train_loader, val_loader

# # Paths and processor
# train_real_dir = "/teamspace/studios/this_studio/audio_detect/dataset/split_data/train/real"
# train_fake_dir = "/teamspace/studios/this_studio/audio_detect/dataset/split_data/train/fake"
# val_real_dir = "/teamspace/studios/this_studio/audio_detect/dataset/split_data/val/real"
# val_fake_dir = "/teamspace/studios/this_studio/audio_detect/dataset/split_data/val/fake"

# # Load processor
# processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")

# # Initialize DataLoaders
# train_loader, val_loader = create_dataloaders(
#     train_real_dir, train_fake_dir, val_real_dir, val_fake_dir, processor, batch_size=16
# )

# # Debug first batch to confirm GPU compatibility
# for batch in train_loader:
#     print("Input shape:", batch["input_values"].shape)
#     print("Attention mask shape:", batch["attention_mask"].shape)
#     print("Labels shape:", batch["labels"].shape)
#     # Example: Move batch to GPU (will be handled by PyTorch Lightning in training)
#     if torch.cuda.is_available():
#         batch = {k: v.to("cuda") for k, v in batch.items()}
#         print("Batch moved to GPU:", batch["input_values"].device)
#     break

In [ ]:
# Run in terminal
# %pip uninstall torchaudio
# %pip install torchaudio --index-url https://download.pytorch.org/whl/cu128
